<h1>Capstone Project</h1>
<p>This notebook is intended as a basis for the IBM Data Science certificate capstone project</p>

In [2]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
